Generate multiple text matrix

In [ ]:
import numpy as np
from numpy import savetxt
from pathlib import Path
import json
from datetime import datetime
import time

from numba import njit, prange

# Create folder
date_time = datetime.now().strftime("%m_%d_%Y_%H_%M")
print(f"Created folder data_{date_time}")

# Remove drive/MyDrive/HPC if it runs on colab only
# p = Path('drive/MyDrive/HPC/data_' + date_time)
p = Path('HPC/data_' + date_time)

p.mkdir(exist_ok=True)

In [ ]:
@njit(parallel=True)
def mat_mult(A, B):
    assert A.shape[1] == B.shape[0]
    res = np.zeros((A.shape[0], B.shape[1]), )
    for i in prange(A.shape[0]):
        for k in range(A.shape[1]):
            for j in range(B.shape[1]):
                res[i,j] += A[i,k] * B[k,j]
    return res

In [ ]:
# The function here will restrict the scope, to make sure that it generates not very small number
# Such as: min_range = -100 => -100 to -10
# def random_within_2_ranges_of(min_range, max_range, is_int):
#   if (min_range == 0):
#     return np.random.randint(0, max_range) if is_int else np.random.uniform(0, max_range)

#   if (max_range == 0):
#     return np.random.randint(min_range, 0) if is_int else np.random.uniform(min_range, 0)

#   arr1 = np.random.randint(min_range,min_range/10) if is_int else np.random.uniform(min_range,min_range/10)
#   arr2 = np.random.randint(max_range/10,max_range) if is_int else np.random.uniform(max_range/10,max_range)
#   out = np.stack((arr1,arr2))
#   out = np.random.choice(out)
#   return out

# random_within_2_ranges_of(-11, 0, False)

In [ ]:
def generate_matrix(folder, test_no, max_dimension, lowest_val, highest_val, is_int):
  n = np.random.randint(max_dimension/10, max_dimension)
  m = np.random.randint(max_dimension/10, max_dimension)

  start_time = time.time()
  a = np.random.randint(lowest_val, highest_val, (n, m)) if is_int else np.random.uniform(lowest_val, highest_val, (n, m))
  b = np.random.randint(lowest_val, highest_val, (m, n)) if is_int else np.random.uniform(lowest_val, highest_val, (m, n))
  end_time = time.time()
  print(f'Init {test_no} took {format(end_time - start_time)}')

  start_time = time.time()
  # c = np.dot(a, b)
  # c = mat_mult(a, b)
  # c = np.asarray(0)
  end_time = time.time()

  desc = f'TestNo {test_no} C=A.B with {n}x{m} dot {m}x{n}, Colab time {format(end_time - start_time)}, is_all_integers {is_int}, highest {highest_val}, lowest {lowest_val}'
  print(desc)

  # data = {'Formular': desc,
  #         'A': str(a.tolist()), 
  #         'B': str(b.tolist()), 
  #         'C': str(c.tolist())}

  start_time = time.time()
  # with open(f'{folder}/TestNo_{test_no}_{n}x{m}_{is_int}_{highest_val}_{lowest_val}.json', 'w', encoding='utf-8') as f:
  #     json.dump(data, f, ensure_ascii=False, indent=4)

  filename = f'{folder}/TestNo_{test_no}_{n}x{m}_{is_int}_{highest_val}_{lowest_val}.txt'

  with open(filename, 'w') as f:
    f.write(desc)
    for line in a:
        np.savetxt(f, line, fmt='%.2f', newline= ' ')
    for line in b:
        np.savetxt(f, line, fmt='%.2f', newline= ' ')
    # for line in c:
    #     np.savetxt(f, line, fmt='%.2f', newline= ' ')

  end_time = time.time()

  print(f'Wrote {test_no} took {format(end_time - start_time)} \n---')


In [ ]:
def generate_list_test(lst):
  counter = 0
  for is_int in [True, False]:
    for item in lst:
      for i in range(item['test_files_no']):
        counter += 1
        generate_matrix(p, counter, item['max_dim'], item['min_val'], item['max_val'], is_int)


In [ ]:
lst = [{'test_files_no': 10, 'max_dim': 10, 'min_val': 0, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': 0, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': 0, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': 0, 'max_val': 10000},

       {'test_files_no': 10, 'max_dim': 10, 'min_val': -10, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': -100, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': -1000, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': -10000, 'max_val': 0},

       {'test_files_no': 10, 'max_dim': 10, 'min_val': -10, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': -100, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': -1000, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 10, 'min_val': -10000, 'max_val': 10000},

       {'test_files_no': 10, 'max_dim': 100, 'min_val': 0, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': 0, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': 0, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': 0, 'max_val': 10000},

       {'test_files_no': 10, 'max_dim': 100, 'min_val': -10, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': -100, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': -1000, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': -10000, 'max_val': 10000},

       {'test_files_no': 10, 'max_dim': 100, 'min_val': -10, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': -100, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': -1000, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 100, 'min_val': -10000, 'max_val': 0},

       {'test_files_no': 10, 'max_dim': 1000, 'min_val': 0, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': 0, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': 0, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': 0, 'max_val': 10000},

       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -10, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -100, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -1000, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -10000, 'max_val': 10000},

       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -10, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -100, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -1000, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 1000, 'min_val': -10000, 'max_val': 0},

       {'test_files_no': 10, 'max_dim': 10000, 'min_val': 0, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': 0, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': 0, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': 0, 'max_val': 10000},
       
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -10, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -100, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -1000, 'max_val': 0},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -10000, 'max_val': 0},

       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -10, 'max_val': 10},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -100, 'max_val': 100},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -1000, 'max_val': 1000},
       {'test_files_no': 10, 'max_dim': 10000, 'min_val': -10000, 'max_val': 10000}]


In [ ]:
generate_list_test(lst)

In [ ]:
1000/10

In [ ]:
mat = np.asarray([[1, 2, 3], [4, 5, 6], [7, 8, 9]])


In [ ]:
mat

In [ ]:
k = np.random.randint(1,4, (10000, 40000))

In [ ]:
k

In [ ]:
with open('outfile.txt', 'w') as f:
    for line in k:
        np.savetxt(f, line, fmt='%.2f')

In [ ]:
np.array2string(k)